In [ ]:
from utils.sample import Sample
from scipy.optimize import brentq
import pandas as pd
import numpy as np
import math
import pickle
import scipy.stats as st
import matplotlib.pyplot as plt
from utils.biseccion import bisec
from utils.black_scholes import raiz_ratio, raiz, call_price, d1
from sklearn.metrics import r2_score

In [ ]:
i = 0
sampl = []
result = []
while i < 10**5:
    S = np.random.uniform(50, 100)
    K = np.random.uniform(S/2, S*2)
    T = np.random.uniform(0.2, 1.1)
    r = np.random.uniform(0.02, 0.1)
    o = np.random.uniform(0.01, 1)
    c = call_price(S, K, r, o, T)
    if c > 0:
        result.append(o)
        sampl.append([c , S, K, r, T])

        i += 1

In [ ]:
df = pd.DataFrame(sampl, columns=['c', 'S', 'k', 'r', 'T'])
df['o'] = result
df.head()

In [ ]:
# Tiro los casos en que no se puede aplicar el metodo de biseccion
# y calculo la volatilidad implícita
vol_bisec = []
vol_brent = []
i = 0
drops = []
for c, S, k, r, T in sampl:
    f = lambda x: raiz(c, S, k, r, x, T)
    # que se cumpla la precondicción
    if f(0.01) < 0:
        # máxima precision
        o_bic = bisec(f, 0.01, 1, 2**-56)
        o_bren = brentq(f, 0.01, 1, xtol=2**-56)
        vol_bisec.append(o_bic)
        vol_brent.append(o_bren)
    else: 
        drops.append(i)
    i += 1
    
    if i % 10000 == 0:
        print('{}%'.format(100*i/len(sampl)))
    
    
dfb = df.drop(drops)
#volatilidad implicita
dfb['o_bis'] = vol_bisec
dfb['o_bren'] = vol_brent

In [ ]:
# error entre la volatilidad estimada e implícita
dfb['diff_bis'] = (dfb['o'] - dfb['o_bis']).apply(abs)
dfb['diff_bren'] = (dfb['o'] - dfb['o_bren']).apply(abs)
print('Error cuadratico medio biseccion: ', np.square(dfb['o'] - dfb['o_bis']).mean())
print('Error absoluto medio biseccion: ', dfb['diff_bis'].mean())
aux = 100*(np.abs(dfb['o'] - dfb['o_bis']) / dfb['o']).mean() 
print('Error absoluto porcentual medio biseccion', aux)
print('r2 Biseccion', r2_score(dfb['o'], dfb['o_bis']))

print('\n\n')

print('Error cuadratico medio Brent: ', np.square(dfb['o'] - dfb['o_bren']).mean())
print('Error absoluto medio Brent: ', dfb['diff_bren'].mean())
aux = 100*(np.abs(dfb['o'] - dfb['o_bren']) / dfb['o']).mean() 
print('Error absoluto porcentual medio Brent', aux)
print('r2 Brent', r2_score(dfb['o'], dfb['o_bren']))

In [ ]:
#Ordeno el dataframe segun el error entre la volatilidad estimada
# y la volatilidad implícita del método de bisección
orda = dfb.sort_values('diff_bis', ignore_index=True, ascending= False)

In [ ]:
#aplico la función que busca la raiz sobre la volatilidad generada
#mediante el método de bisección
fs = []
auxi = orda[['c','S', 'k', 'r', 'o_bis', 'T']]
for i in range(len(auxi)):
    fs.append(raiz(*auxi.iloc[i]))

orda['f(o_bis)'] = fs


In [ ]:
orda.head()

Observar q en muchos casos la diferencia entre la volatilidad implícita y la volatilidad estimada es grande, eso es porque en ciertos casos la volatilidad tiene poco impacto, osea un "vega bajo", y por el problema de precisión la función que calcula la raíz retorna 0.

In [ ]:

#vega en funcion de ratio. La función retorna vega/K.
def vega(S, K, r, o, T):
    
    d1 = (np.log(S/K) + (r + 0.5 * o ** 2) * T) / (o * np.sqrt(T))
    
    vega = np.sqrt(T) * S * math.exp(-d1**2/2) / np.sqrt(2*math.pi)
    
    return vega

In [ ]:
SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

fi_1 = '\u03A61'.translate(SUB)+'(o)'
fi_2 = '\u03A62'.translate(SUB)+'(o)'
fi_1_b = '\u03A61'.translate(SUB)+'(o_bis)'
fi_2_b = '\u03A62'.translate(SUB)+'(o_bis)'
fi_1_01 = '\u03A61'.translate(SUB)+'(0.01)'
fi_2_01 = '\u03A62'.translate(SUB)+'(0.01)'

fi_1, fi_2, fi_1_01, fi_2_01

In [ ]:
def my_vega(elem):
    return vega(elem['S'],elem['k'], elem['r'], elem['o'], elem['T'])

d_1 = d1(*orda[['S','k','r','o','T']].iloc[0])
#d_2 = d_1 - orda.iloc[0]['o']*math.sqrt(orda.iloc[0]['T']) 
d_2 = d_1 - 0.01*math.sqrt(orda.iloc[0]['T']) 
norm1 = st.norm.cdf(d_1)
d_1_p =d1(*orda[['S','k','r']].iloc[0],0.01,orda.iloc[0]['T'])
st.norm.cdf(d_1), st.norm.cdf(d_2)

def my_norm_1(elem):
    d_1 = d1(elem['S'],elem['k'], elem['r'], elem['o'], elem['T'])
    return st.norm.cdf(d_1)

def my_norm_2(elem):
    d_1 = d1(elem['S'],elem['k'], elem['r'], elem['o'], elem['T'])
    return st.norm.cdf(d_1 - elem['o']*math.sqrt(elem['T']))

def my_norm_1_b(elem):
    d_1 = d1(elem['S'],elem['k'], elem['r'], elem['o_bis'], elem['T'])
    return st.norm.cdf(d_1)

def my_norm_2_b(elem):
    d_1 = d1(elem['S'],elem['k'], elem['r'], elem['o_bis'], elem['T'])
    return st.norm.cdf(d_1 - elem['o']*math.sqrt(elem['T']))

def my_norm_1_001(elem):
    d_1 = d1(elem['S'],elem['k'], elem['r'], 0.01, elem['T'])
    return st.norm.cdf(d_1)

def my_norm_2_001(elem):
    d_1 = d1(elem['S'],elem['k'], elem['r'], 0.01, elem['T'])
    return st.norm.cdf(d_1 - 0.01*math.sqrt(elem['T']))


orda['vega'] = orda.apply(my_vega, axis=1)
orda[fi_1] = orda.apply(my_norm_1, axis=1)
orda[fi_2] = orda.apply(my_norm_2, axis=1)
orda[fi_1_01] = orda.apply(my_norm_1_001, axis=1)
orda[fi_2_01] = orda.apply(my_norm_2_001, axis=1)
orda[fi_1_b] = orda.apply(my_norm_1_b, axis=1)
orda[fi_2_b] = orda.apply(my_norm_2_b, axis=1)


In [ ]:
orda.head(50)

In [ ]:
#Los que tienen mas error cumplen esa condicion
#Obsevar en la celda siguiente a esta que hay vetas mucho menor
# a este ejemplo, y sin embargo tiene un error mucho mas grande
print(fi_1 + ': {}'.format(orda.iloc[0][fi_1]))
print(fi_2 + ': {}'.format(orda.iloc[0][fi_2]))
print(fi_1_b + ': {}'.format(orda.iloc[0][fi_1_b]))
print(fi_2_b + ': {}'.format(orda.iloc[0][fi_2_b]))

print(fi_1_01 + ': {}'.format(orda.iloc[0][fi_1_01]))
print(fi_2_01 + ': {}'.format(orda.iloc[0][fi_2_01]))
print('f(o_bis): ', orda['f(o_bis)'].iloc[0])

In [ ]:

with open('dataFrame/orda.pickle', 'wb') as handle:
    pickle.dump(orda, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
ordy = pd.read_csv('dataFrame/orda.csv')
ordy.head()

In [ ]:
#Observar q los casos con mas error son ITM
#Fitro por ratio < 1 OTM
#NOTAR QUE HAY CASOS CON VEGA EXTREMADAMENTE CHICO PERO SU ERROR NO ES GRANDE
orda[orda['S']/orda['k'] < 1].head(50)


In [ ]:
#porcentaje de elementos en que no se encontro la raiz
# mediante el método de bisección con tolerancia 2**-56
100*len(orda[orda['f(o_bis)'] != 0])/len(orda)
#quiere decir q no termina, si no que corta a las 100 iteraciones


In [ ]:
ordas = orda.sort_values('vega', ignore_index=True)

In [ ]:
auxi = ordas[(ordas['o'] < 0.52) & (ordas['o'] > 0.48)]
vega_chico = auxi.iloc[0]
vega_grande = auxi.iloc[-1]
vega_mediano = auxi.iloc[len(auxi)//2]
vega_new = auxi.iloc[len(auxi)//4]
print('Vega Chico: ', vega_chico['vega'])
print('Vega Mediano: ', vega_mediano['vega'])
print('Vega Grande: ', vega_grande['vega'])


In [ ]:
def my_call_price(o, elem=vega_chico):
    return call_price(elem['S'], elem['k'], elem['r'], o, elem['T'])

sdvc = np.vectorize(my_call_price)

f = lambda x: my_call_price(x, elem=vega_grande)
sdvg = np.vectorize(f)

m = lambda x: my_call_price(x, elem=vega_mediano)
sdvm = np.vectorize(m)

In [ ]:
fig, ax, = plt.subplots(figsize=(15,10))
volatilidad = np.linspace(0.001, 2, 1000)
#ax.plot(history_clr.history['loss'], 'b', label = 'implied volatility')
my_round = lambda x: round(x, 4)
ax.plot(volatilidad, sdvc(volatilidad),'m',
    label='Vega Chico S: {}, K: {}, r: {}, T: {}, vega(o=0.5): {}'.format(*vega_chico[['S','k', 'r', 'T', 'vega']].apply(my_round)))

######################################################
ax.plot(volatilidad, sdvm(volatilidad),'k',
    label='Vega Mediano S: {}, K: {}, r: {}, T: {}, vega(o=0.5): {}'.format(*vega_mediano[['S','k', 'r', 'T', 'vega']].apply(my_round)))
######################################################
ax.plot(volatilidad, sdvg(volatilidad),'b',
    label='Vega Grande S: {}, K: {}, r: {}, T: {}, vega(o=0.5): {}'.format(*vega_grande[['S','k', 'r', 'T', 'vega']].apply(my_round)))
#########################################################

ax.legend()
plt.legend(fontsize=14)
plt.title('Comparacion vega vs volatilidad')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel("volatilidad", fontsize=18)
plt.ylabel("precio call", fontsize=18)
plt.grid()
plt.savefig('volatilidad_vega',dpi=300, bbox_inches='tight')
plt.show()